In  a docker terminal run
jupyter notebook --ip=0.0.0.0 --port=8888 --no-browser --allow-root

then open jupyter in the printed link (such as http://127.0.0.1:8888/tree?token=4d9d578397aede50fc5bd2d92794b562a6bbcbc73b2dfe51)

CODE HERE, REFRESH AND EXECUTE IN THE BROWSER

In [1]:
import os
import sys
import django

os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"


In [2]:
sys.path.append('/app')

os.environ['DJANGO_SETTINGS_MODULE'] = 'app.settings'
os.environ['PYTHONPATH'] = '/app'

django.setup()

In [3]:
from toolbox import models
from toolbox import views
from django.forms.models import model_to_dict
import numpy as np
import json
from django.contrib.gis.geos import GEOSGeometry, Polygon, MultiPolygon
from django.db.models import Q
from buek import models as b_m

In [4]:
user_field_id = 63
pr = {
    "id": None,
    "userField": 63,
    "gek_priority": "4",
    "gek_costs_max": "3715774",
    "gek_costs_min": "898036",
    "gek_landuse": [
        "52",
        "53",
        "64"
    ],
    "all_sieker_gek_ids": [
        5,
        6,
        90,
        2,
        3,
        83,
        118,
        117,
        119
    ],
    "selected_sieker_geks": [],
    "all_sieker_gek_measure_ids": [],
    "selected_sieker_gek_measures": []
}

In [5]:
project = {
    "id": None,
    "userField": 63,
    "gek_priority": "8",
    "gek_landuse": [
        "53",
        "64"
    ],
    "all_sieker_gek_ids": [
        5,
        6,
        90,
        2,
        3,
        83,
        118,
        117,
        119
    ],
    "selected_sieker_geks": [
        5,
        6,
        90,
        2,
        3,
        83,
        118,
        117,
        119
    ],
    "all_sieker_gek_measure_ids": [],
    "selected_sieker_gek_measures": [],
    "gek_costs_min": "898036",
    "gek_costs_max": "3715774"
}

In [6]:
user_field = models.UserField.objects.get(pk=63)

In [7]:
geks = models.GekRetention.objects.filter(Q(geom4326__intersects=user_field.geom) | Q(geom4326__within=user_field.geom))

In [8]:
geks.values_list('id', flat=True)


<QuerySet [5, 6, 90, 2, 3, 83, 118, 117, 119]>

In [9]:
landuses = models.GekLanduse.objects.filter(Q(gek_retention__in=geks) & Q(clc_landuse__id__in=project['gek_landuse']))

In [10]:
landuses.values_list()

<QuerySet [(59, 6, 'Laubwald, Wiesen, Ackerland, städt. Prägung', '11', 64, 36105.1371442056, 3477.4719281814464, 0.09631515632504764), (35, 6, 'Laubwald, Wiesen, Ackerland, städt. Prägung', '31', 53, 36105.1371442056, 7544.156886476218, 0.2089496809372169), (52, 2, 'Ackerland, Grünfläche', '11', 64, 122476.1799980161, 3754.318534756101, 0.03065345877718357), (272, 83, 'Wiese und Weide', '11', 64, 7630.582087049749, 17.36292581515837, 0.0022754392282373684), (257, 117, 'Wiese und Weide, Acker', '31', 53, 9499.855094449507, 116.67765531726569, 0.012282045795144506), (29, 117, 'Wiese und Weide, Acker', '11', 64, 9499.855094449507, 97.7688154853822, 0.010291611241786805), (206, 119, 'Wiese und Weide', '11', 64, 10835.980648107761, 1741.6380917242723, 0.1607273165468791)]>

In [11]:
geks = models.GekRetention.objects.filter(landuses__in=landuses).distinct()

In [12]:
geks.values_list('id', flat=True)

<QuerySet [2, 6, 83, 117, 119]>

In [13]:
project['gek_landuse']

['53', '64']

In [14]:
def add_range_filter(filters, obj, field,  model_field=None):
    print(obj)
    model_field = model_field or field
    min_val = obj.get(f'{field}_min')
    max_val = obj.get(f'{field}_max')

    print(min_val, max_val)

    if min_val is not None:
        if model_field == 'index_soil':
            min_val = float(min_val) / 100
        else:
            min_val = float(min_val)
        filters &= Q(**{f"{model_field}__gte": min_val})
    if max_val is not None:
        if model_field == 'index_soil':
            max_val = float(max_val) / 100
        else:
            max_val = float(max_val)
        filters &= Q(**{f"{model_field}__lte": max_val})

    return filters

In [15]:
Q()

<Q: (AND: )>

In [16]:
filters = Q(priority__priority_level__gte=project['gek_priority'])

In [17]:
filters = add_range_filter(filters, project, 'gek_costs', 'costs')

{'id': None, 'userField': 63, 'gek_priority': '8', 'gek_landuse': ['53', '64'], 'all_sieker_gek_ids': [5, 6, 90, 2, 3, 83, 118, 117, 119], 'selected_sieker_geks': [5, 6, 90, 2, 3, 83, 118, 117, 119], 'all_sieker_gek_measure_ids': [], 'selected_sieker_gek_measures': [], 'gek_costs_min': '898036', 'gek_costs_max': '3715774'}
898036 3715774


In [17]:
measures = models.GekRetentionMeasure.objects.filter(
            gek_retention__in=geks
        )

In [18]:
measures.count()

20

In [18]:
measures = models.GekRetentionMeasure.objects.filter(
            gek_retention__in=geks
        ).filter(filters)

In [19]:
measures.values_list()

<QuerySet [(157, 6, 72, 3.0, 'Totholz zur Strömungs- und Substratdiversität fest einbauen, Initialgerinne für Neutrassierung', 1, '2.775.000', 2775000, 4)]>

In [20]:
geks = models.GekRetention.objects.filter(measures__in=measures).distinct()

In [21]:
geks.values_list()

<QuerySet [(6, <Polygon object at 0x7faa04a05f10>, <Polygon object at 0x7faa04a05f90>, <Point object at 0x7faa04a06010>, 'Lichtenower Mühlenfließ', 'Laubwald, Wiesen, Ackerland, städt. Prägung', 'GEK', 'DEM', 'Wasser- und Bodenverband Stöbber-Erpe', 'LMF_05', 'Bundesanstalt für Gewässerkunde', 2, 15, '20.11.2024')]>

In [22]:
measures.filter(gek_retention=geks[0])


<QuerySet [<GekRetentionMeasure: GekRetentionMeasure object (157)>]>

In [23]:
measures[0].to_dict(language='de')

{'gek_measure': 'Maßnahmen zur Habitatverbesserung im Gewässer durch Laufveränderung, Ufer- oder Sohlgestaltung inkl. Begleitender Maßnahmen',
 'quantity': 3.0,
 'description_de': 'Totholz zur Strömungs- und Substratdiversität fest einbauen, Initialgerinne für Neutrassierung',
 'priority': 1,
 'kosten': '2.775.000',
 'costs': 2775000,
 'measure_number': 4}

In [25]:
geks[0].to_dict()

{'name': 'Lichtenower Mühlenfließ',
 'quelle_1': 'GEK',
 'quelle_2': 'DEM',
 'association': 'Wasser- und Bodenverband Stöbber-Erpe',
 'planning_segment': 'LMF_05',
 'hrsg': 'Bundesanstalt für Gewässerkunde',
 'gek_document': 'https://www.wasserblick.net/servlet/is/134504/',
 'number_of_measures': 15}

In [26]:
[m.to_dict() for m in measures]

[{'gek_measure': 'Maßnahmen zur Habitatverbesserung im Gewässer durch Laufveränderung, Ufer- oder Sohlgestaltung inkl. Begleitender Maßnahmen',
  'quantity': 3.0,
  'description_de': 'Totholz zur Strömungs- und Substratdiversität fest einbauen, Initialgerinne für Neutrassierung',
  'priority': 1,
  'kosten': '2.775.000',
  'costs': 2775000,
  'measure_number': 4}]

In [29]:
[m.to_dict() for m in measures if m.gek_retention == geks[0]] 

[{'gek_measure': 'Maßnahmen zur Habitatverbesserung im Gewässer durch Laufveränderung, Ufer- oder Sohlgestaltung inkl. Begleitender Maßnahmen',
  'quantity': 3.0,
  'description_de': 'Totholz zur Strömungs- und Substratdiversität fest einbauen, Initialgerinne für Neutrassierung',
  'priority': 1,
  'kosten': '2.775.000',
  'costs': 2775000,
  'measure_number': 4}]